In [1]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [2]:
import kagglehub

path = kagglehub.dataset_download('aashita/nyt-comments')
print(path)

c:\Users\Playdata2\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\Playdata2\.cache\kagglehub\datasets\aashita\nyt-comments\versions\13


In [3]:
from glob import glob
article_lists = glob(path+'/*.*',recursive=True)

In [4]:
import pandas as pd
pd.read_csv(article_lists[0]).head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


##### LSTM
- 입력 : "나는 파이썬을 좋아합니다. 따라서 나는 ___ 을 잘합니다."
- 일반신경망 : 공부  ( 파이썬 정보가 희석... 잊어)
- LSTM : 프로그래밍(오래된 정보도 기억)

- 핵심 키워드
    - 장기기억 : 중요한 정보는 오래기억
    - 단기기억 : 불필요한 정보는 버림
    - 순서이해 : 시간순서 이해

##### 3개의 Gate를 통해 정보의 흐름을 제어
 - LSTM 셀 (한 시점 t)
    - 입력 : $x_t$ (현재데이터)
    - 이전 은닉상태 : $h_{t-1}$
    - 이전 셀 상태 : $c_{t-1}$

    --> forget gate --> input date   --> output gate
        잊을데이터        추가할 데이터     출력할 데이터
    
    출력 $h_t$ ,  $c_t$

Forget Gate(잊음 관문)


$f_t$ = $s(w_f . [h_{t-1}, x_t ] + b_f )$

s : sigmoid함수(0~1)

이전 셀상태 : [1.5, -0.3, 2.1]
현재입력 : '새로운 문장 입력'

$f_t$ = [0.1,0.05,0.9]

결과 : [ 1.5*0.1, -0.3*0.05, 2.1*0.9  ]  =  [0.15, -0.015, 1.89]

첫 2개는 버리고 3번째는 유지


##### input gate : 입력
- 첫 번째는 70%받고 두번째는 30% 받음

##### Cell State 업데이트
 - 이전기억에서 필요한 것만 유지하고 새로운 정보에서 필요한 것만 유지


input 게이트

시점 t에서:
- $x_t$      : 현재 입력 데이터 (벡터)
- $h_{t-1}$  : 이전 시점의 은닉 상태 (벡터)
- $C_{t-1}$  : 이전 시점의 셀 상태 (벡터) ← 장기 기억!
- $W_*$, $U_*$ : 가중치 행렬
- $b_*$      : 편향 벡터

- $x_t$ = [0.2, -0.5, 0.8]      (3개 입력 피처)
- $h_{t-1}$ = [0.1, 0.3, -0.2, 0.5]  (4개 은닉)
- $C_{t-1}$ = [0.4, -0.1, 0.6, 0.2]  (4개 셀 상태)

Forget Gate ( $f_t$ ) - 어제 기억을 얼마나 유지할까
- $f_t$ = $σ( W_f · [h_{t-1}, x_t] + b_f )$

1단계: $h_{t-1}$ 과 $x_t$ 를 연결 (concatenate)

   $[ h_{t-1}, x_t]$ = [0.1,    0.3,    -0.2,    0.5,    0.2,    -0.5,    0.8]

               $h_{t-1}$    $x_t$

                        4개              3개

                               ↓

                           총 7개 벡터

2단계: 가중치 행렬 곱하기
   $W_f · [h_{t-1}, x_t]$
   
   $W_f$ 는 크기: (4, 7) 행렬
   (왜 (4, 7)? → 은닉 크기 4개, 입력 7개)
   
   결과: 4개의 값

3단계: 편향 더하기
   + $b_f$  (크기: 4)
   
   결과: 4개의 값

4단계: Sigmoid 함수 적용
   $σ(x)$ = $\frac{1}{1 + e^{-x}}$ 
   
   이 함수는 모든 값을 0~1 사이로 변환!
   
   $f_t = [0.3, 0.8, 0.1, 0.9]$
   
   의미:
   - 첫 번째 셀 상태: 30% 유지 (70% 잊음)
   - 두 번째 셀 상태: 80% 유지 (20% 잊음)
   - 세 번째 셀 상태: 10% 유지 (90% 잊음)
   - 네 번째 셀 상태: 90% 유지 (10% 잊음)

#### 학습용 데이터셋 준비하기

##### 데이터 전처리
- 문장단위로 분류
- 특수문자 제거
- 소문자 변환

In [5]:
all_headline = []
articles = [path for path in article_lists if 'Articles' in path] # ^
# headline 정보만 추출
# 전처리 : 소문자로 변경하고 특수문자 제거
# 특수문자 제거
import string
string.punctuation # ^ 파이썬의 특수문자 모음

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
df = pd.read_csv(article_lists[0])
cleaned_sentence = [doc.lower() for doc in df.headline.values if doc not in string.punctuation]

# 모든 문장의 단어를 추출해 고유 번호 지정
bow = {}
for line in cleaned_sentence:
    for w in line.split():
        if w not in bow:
            bow[w] = len(bow.keys())
bow

{'finding': 0,
 'an': 1,
 'expansive': 2,
 'view': 3,
 'of': 4,
 'a': 5,
 'forgotten': 6,
 'people': 7,
 'in': 8,
 'niger': 9,
 'and': 10,
 'now,': 11,
 'the': 12,
 'dreaded': 13,
 'trump': 14,
 'curse': 15,
 'venezuela’s': 16,
 'descent': 17,
 'into': 18,
 'dictatorship': 19,
 'stain': 20,
 'permeates': 21,
 'basketball': 22,
 'blue': 23,
 'blood': 24,
 'taking': 25,
 'things': 26,
 'for': 27,
 'granted': 28,
 'caged': 29,
 'beast': 30,
 'awakens': 31,
 'ever-unfolding': 32,
 'story': 33,
 'o’reilly': 34,
 'thrives': 35,
 'as': 36,
 'settlements': 37,
 'add': 38,
 'up': 39,
 'mouse': 40,
 'infestation': 41,
 'divide': 42,
 'g.o.p.': 43,
 'now': 44,
 'threatens': 45,
 'tax': 46,
 'plan': 47,
 'variety': 48,
 'puzzle:': 49,
 'acrostic': 50,
 'they': 51,
 'can': 52,
 'hit': 53,
 'ball': 54,
 '400': 55,
 'feet.': 56,
 'but': 57,
 'play': 58,
 'catch?': 59,
 'that’s': 60,
 'tricky.': 61,
 'country,': 62,
 'shock': 63,
 'at': 64,
 'budget': 65,
 'cuts': 66,
 'why': 67,
 'is': 68,
 'this': 6

##### 만든 문장을 토크나이저 하기

In [7]:
# 예시
[bow[w] for w in cleaned_sentence[10].split()], cleaned_sentence[10]

([48, 49, 50], 'variety puzzle: acrostic')

In [8]:
import numpy as np
from glob import glob

from torch.utils.data.dataset import Dataset

# kaggle에서 데이터 download
import kagglehub
path = kagglehub.dataset_download('aashita/nyt-comments')

# csv 파일이 있는 경로 path
# csv의 절대경로 리스트
csv_lists = glob(path+'/*.*')

class TextGeneration(Dataset):
    def clean_text(self, txt):
        # 모든 단어를 소문자로 바꾸고 특수문자를 제거
        txt = "".join(v for v in txt if v not in string.punctuation).lower()
        return txt
    def __init__(self,csv_lists):
        all_headlines = []

        # 모든 헤드라인의 텍스트를 불러옴
        for filename in csv_lists:
            if 'Articles' in filename:
                article_df = pd.read_csv(filename)

                # 데이터셋의 headline의 값을 all_headlines에 추가
                all_headlines.extend(list(article_df.headline.values)) # ^ [[]] 형태가 안되도록, append
                break

        # headline 중 unknown 값은 제거
        all_headlines = [h for h in all_headlines if h != "Unknown"]

        # 구두점 제거 및 전처리가 된 문장들을 리스트로 반환
        self.corpus = [self.clean_text(x) for x in all_headlines]
        self.BOW = {}

        # 모든 문장의 단어를 추출해 고유번호 지정
        for line in self.corpus:
            for word in line.split():
                if word not in self.BOW.keys():
                    self.BOW[word] = len(self.BOW.keys())

        # 모델의 입력으로 사용할 데이터
        self.data = self.generate_sequence(self.corpus)
    def generate_sequence(self, txt):
        seq = []

        for line in txt:
            line = line.split()
            line_bow = [self.BOW[word] for word in line]

            # 단어 2개를 입력으로, 그 다음 단어를 정답으로
            data = [([line_bow[i], line_bow[i+1]], line_bow[i+2]) 
            for i in range(len(line_bow)-2)]
            
            seq.extend(data)

        return seq
    def __len__(self):
        return len(self.data)
    def __getitem__(self, i):
        data = np.array(self.data[i][0])  # 입력 데이터
        label = np.array(self.data[i][1]).astype(np.float32)  # 출력 데이터

        return data, label

In [9]:
dataset = TextGeneration(csv_lists)
len(dataset)
x, y  = next(iter(dataset))
x, y

(array([0, 1]), array(2., dtype=float32))

In [10]:
#데이터가 2개의 array로 만들어지는 과정 정리하기

##### LSTM 모델 정의

In [13]:
import torch
import torch.nn as nn
class LSTM(nn.Module):
    def __init__(self,num_embeddings): # num_embeddings 전체 단어의 개수(어휘사전 크기)
        super(LSTM, self).__init__()
        # 신경망이 이해할 수 있도록 벡터로 변경
        self.embed = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=16)
        
        # LSTM을 5개 층 (배치, 시퀀스, 피처) 16 ~ 512
        self.lstm = nn.LSTM(input_size=16,hidden_size=64,num_layers=5,batch_first=True)
        # 분류를 위한 fc 층
        self.fc1 = nn.Linear( 128 , num_embeddings)
        self.fc2 = nn.Linear(num_embeddings ,num_embeddings)
        self.relu = nn.ReLU()
    def forward(self,x): # 입력 (배치크기 , sequence_length) batch : 32, sq_len : 2
                         # 출력 (배치크기, sequence_length, 16) 32 2 16
        x = self.embed(x)

        # lstm 모델 예측값
        x, _ = self.lstm(x) # 츨력 (batch, sq_len, 64) 32 2 64
        x = torch.reshape(x,(x.shape[0],-1)) # 출력 (batch, sq_len x 64) 32, 128
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
        return out

In [14]:
# 모델...
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# GPU가 사용 가능한 환경이면 GPU를 쓰고, 아니면 CPU를 쓰겠다
dataset = TextGeneration(csv_lists)
model = LSTM(num_embeddings=len(dataset.BOW)).to(device)
loader = DataLoader(dataset,batch_size=64)
optim = Adam(model.parameters(), lr=1e-3)

for epoch in range(200):
    loop = tqdm(loader)
    for data,label in loop:
        data, label = data.to(device), label.to(device)
        optim.zero_grad()
        pred = model(torch.tensor(data, dtype=torch.long))
        loss = nn.CrossEntropyLoss()(pred,torch.tensor(label,dtype=torch.long))
        loss.backward()
        optim.step()
        loop.set_description(f'epoch : {epoch+1}loss : {loss.item()}')

  0%|          | 0/63 [00:00<?, ?it/s]C:\Users\Playdata2\AppData\Local\Temp\ipykernel_17260\3789114445.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = model(torch.tensor(data, dtype=torch.long))
C:\Users\Playdata2\AppData\Local\Temp\ipykernel_17260\3789114445.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = nn.CrossEntropyLoss()(pred,torch.tensor(label,dtype=torch.long))
epoch : 200loss : 0.21958789229393005: 100%|██████████| 63/63 [00:04<00:00, 12.60it/s]


In [15]:
import torch
# 문장을 예측
# 입력문 장을 텐서로 변경 임베딩 벡터 Bow를 이용해서
sample = 'i love'
with torch.no_grad():
    words = torch.tensor(
        [dataset.BOW[w] for w in sample.split()], dtype=torch.long
    ).to(device).unsqueeze(0)
    # (2,) --> (batch,sq_len) (1,2)
    output = model(words)
    
    # 출력은 단어의 개수만큼 len(BOW) (batch, len(BOW))
    # 확률이 가장 높은 단어 찾기
    predicted_index = torch.argmax(output,dim=1).item()

    # 단어사전 BOW에서 인덱스에 해당하는 단어를 찾기
    # 역 dict를 만들어서 찾기

    reverse_bow = {value:key for key,value in dataset.BOW.items()}
    predicted_word = reverse_bow[predicted_index]   
    print(predicted_word)

my


In [ ]:
# 인덱스 → 값 찾는 법 ^
# https://chatgpt.com/s/t_6903268b21fc819195c5a4dc440d3233